In [86]:
alph = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
# alph = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
# alph = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя"

len_alph = len(alph)

In [87]:
class Playfair:
    def __init__(self, breakdown: str = 'Х', matrix_size: str = (4, 8)):
        self.breakdown = breakdown
        self.matrix: list[str] = list()
        self.matrix_size = matrix_size

    def set_matrix(self, phrase: str) -> None:
        phrase_set = set(phrase)
        ost_alph = "".join([i for i in alph if i not in phrase_set])
        ost = phrase + ost_alph
        
        for i in range(self.matrix_size[1]):
            self.matrix.append(ost[i*self.matrix_size[0]:(i+1)*self.matrix_size[0]])
        self.print_matrix()
        
    def print_matrix(
        self,
        marked: list[list[tuple[int, int]]] | None = None,
    ) -> None:
        matrix_to_print = []
        for line in self.matrix:
            matrix_to_print.append([f" {sym} " for sym in line])

        if marked is not None:
            for i, marked_group in enumerate(marked):
                for pixel in marked_group:
                    if i == 0:
                        sym = f"({self.matrix[pixel[0]][pixel[1]]})"
                    else:
                        sym = f"[{self.matrix[pixel[0]][pixel[1]]}]"
                    matrix_to_print[pixel[0]][pixel[1]] = sym

        for line in matrix_to_print:
            print(*line, sep=" ")

    def prepare_data_to_encrypt(self, data: str) -> list[str]:
        result = []
        last = None
        for i in data:
            if last is None:
                last = i
                continue

            if last == i:
                result.append(last + self.breakdown)
                last = self.breakdown
                continue
                
            result.append(last + i)
            last = None
        
        if last is not None:
            result.append(last + self.breakdown)
        
        return result
    
    def _encrypt_bigram(self, bi: str) -> str:
        bi1 = [None, None]
        bi2 = [None, None]

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(bi[0])) != -1:
                bi1 = [i, index_1]
                break
        else:
            print(bi)
            raise Exception("NOT FOUND IN MATRIX")

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(bi[1])) != -1:
                bi2 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")

        if bi1[0] == bi2[0]:
            bi1[1] = (bi1[1] + 1) % self.matrix_size[0]
            bi2[1] = (bi2[1] + 1) % self.matrix_size[0]
        elif bi1[1] == bi2[1]:
            bi1[0] = (bi1[0] + 1) % self.matrix_size[1]
            bi2[0] = (bi2[0] + 1) % self.matrix_size[1]
        else:
            bi1, bi2 = (bi2[0], bi1[1]), (bi1[0], bi2[1])
        return self.matrix[bi1[0]][bi1[1]] + self.matrix[bi2[0]][bi2[1]]
    
    def encrypt(self, data: str) -> str:
        if not self.matrix:
            raise Exception("NO DATA IN MATRIX")
        prepared_data = self.prepare_data_to_encrypt(data)
        encrypted_data = ""
        for bigram in prepared_data:
            encrypt_bigram = self._encrypt_bigram(bigram)
            encrypted_data += encrypt_bigram
        return encrypted_data
    

    def _visual_encrypting_bigram(self, plain_bi: str) -> str:
        print(f"шифруем биграмму {plain_bi}")
        plain_bi1 = [None, None]
        plain_bi2 = [None, None]

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(plain_bi[0])) != -1:
                plain_bi1 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")

        for i, line in zip(range(len(self.matrix)), self.matrix):
            if (index_1 := line.find(plain_bi[1])) != -1:
                plain_bi2 = [i, index_1]
                break
        else:
            raise Exception("NOT FOUND IN MATRIX")
        
        crypted_bi = None
        crypted_bi1 = plain_bi1.copy()
        crypted_bi2 = plain_bi2.copy()

        if plain_bi1[0] == plain_bi2[0]:
            crypted_bi1[1] = (plain_bi1[1] + 1) % self.matrix_size[0]
            crypted_bi2[1] = (plain_bi2[1] + 1) % self.matrix_size[0]
        elif plain_bi1[1] == plain_bi2[1]:
            crypted_bi1[0] = (plain_bi1[0] + 1) % self.matrix_size[1]
            crypted_bi2[0] = (plain_bi2[0] + 1) % self.matrix_size[1]
        else:
            crypted_bi1, crypted_bi2 = (plain_bi2[0], plain_bi1[1]), (plain_bi1[0], plain_bi2[1])
        crypted_bi = self.matrix[crypted_bi1[0]][crypted_bi1[1]] + self.matrix[crypted_bi2[0]][crypted_bi2[1]]
        
        self.print_matrix(marked = [[plain_bi1, plain_bi2], [crypted_bi1, crypted_bi2]])

        print(f"из '{plain_bi}' получаем '{crypted_bi}'")
        print("----------")
        return crypted_bi
    

    def print_visual(self, data: str):
        if not self.matrix:
            raise Exception("NO DATA IN MATRIX")
        prepared_data = self.prepare_data_to_encrypt(data)
        for bigram in prepared_data:
            self._visual_encrypting_bigram(bigram)




In [88]:
# Демонстрация работы
key = "ЗОЛА"
data = "ПРИВЕТМИРЧТОТО"

playfair_cipher = Playfair()
playfair_cipher.set_matrix(key)

print("шифр Плейфера:")
print(f"{key} - ключ")
print(f"{data} - исходное")
print(f"{playfair_cipher.encrypt(data)} - зашифровано")
print()
playfair_cipher.print_visual(data)

 З   О   Л   А 
 Б   В   Г   Д 
 Е   Ж   И   Й 
 К   М   Н   П 
 Р   С   Т   У 
 Ф   Х   Ц   Ч 
 Ш   Щ   Ъ   Ы 
 Ь   Э   Ю   Я 
шифр Плейфера:
ЗОЛА - ключ
ПРИВЕТМИРЧТОТО - исходное
УКГЖРИЖНФУЛСЛС - зашифровано

шифруем биграмму ПР
 З   О   Л   А 
 Б   В   Г   Д 
 Е   Ж   И   Й 
[К]  М   Н  (П)
(Р)  С   Т  [У]
 Ф   Х   Ц   Ч 
 Ш   Щ   Ъ   Ы 
 Ь   Э   Ю   Я 
из 'ПР' получаем 'УК'
----------
шифруем биграмму ИВ
 З   О   Л   А 
 Б  (В) [Г]  Д 
 Е  [Ж] (И)  Й 
 К   М   Н   П 
 Р   С   Т   У 
 Ф   Х   Ц   Ч 
 Ш   Щ   Ъ   Ы 
 Ь   Э   Ю   Я 
из 'ИВ' получаем 'ГЖ'
----------
шифруем биграмму ЕТ
 З   О   Л   А 
 Б   В   Г   Д 
(Е)  Ж  [И]  Й 
 К   М   Н   П 
[Р]  С  (Т)  У 
 Ф   Х   Ц   Ч 
 Ш   Щ   Ъ   Ы 
 Ь   Э   Ю   Я 
из 'ЕТ' получаем 'РИ'
----------
шифруем биграмму МИ
 З   О   Л   А 
 Б   В   Г   Д 
 Е  [Ж] (И)  Й 
 К  (М) [Н]  П 
 Р   С   Т   У 
 Ф   Х   Ц   Ч 
 Ш   Щ   Ъ   Ы 
 Ь   Э   Ю   Я 
из 'МИ' получаем 'ЖН'
----------
шифруем биграмму РЧ
 З   О   Л   А 
 Б   В   Г   Д 
 Е   Ж   И   